In [1]:
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

In [2]:
data = np.load('./datasets/data512.npz')
signals = data['signals']
labels  = data['labels']
data_num = signals.shape[0]
signals.shape, labels.shape

((10800, 5, 512), (10800,))

In [3]:
# 归一化
scale_signals = np.zeros(signals.shape)
for i in range(5):
    scale_signals[:, i, :] = scale(signals[:, i, :], axis=-1)
print(scale_signals.shape)
print(scale_signals[0])

C:\Users\QI\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


(10800, 5, 512)
[[-0.76792755  2.04493118 -0.76799684 ...  1.15661872 -0.76801259
  -0.13547211]
 [-0.76945589 -0.38509553 -0.76949311 ...  1.44380422 -0.76943418
   0.17152614]
 [-0.78179927  0.5871573  -0.78175805 ... -0.14466428 -0.78130456
   1.88817344]
 [-0.76959921  0.50207479 -0.76960547 ... -0.46163122 -0.76974328
   1.62639848]
 [-0.77726898  2.07317366 -0.77728759 ... -0.39258268 -0.77716971
  -0.1539611 ]]


In [4]:
# FFT
fft_signals = np.abs(np.fft.fft(scale_signals, axis=-1))
print(fft_signals.shape)
print(fft_signals[0])

(10800, 5, 512)
[[0.00000000e+00 5.48108895e+00 1.90332337e+01 ... 5.29012958e+00
  1.90332337e+01 5.48108895e+00]
 [0.00000000e+00 1.29001028e+01 1.74393232e+01 ... 1.29810497e+01
  1.74393232e+01 1.29001028e+01]
 [2.84217094e-14 2.12945171e+01 8.35555201e+00 ... 1.15827241e+01
  8.35555201e+00 2.12945171e+01]
 [0.00000000e+00 1.39388424e+01 1.38523372e+01 ... 7.05243714e+00
  1.38523372e+01 1.39388424e+01]
 [2.84217094e-14 9.40698834e+00 7.69584179e+00 ... 9.46856385e+00
  7.69584179e+00 9.40698834e+00]]


In [5]:
def gaussian(dist, a=1, b=0, c=0.6):
    return a * np.e ** (-(dist - b) ** 2 / (2 * c ** 2))

In [6]:
acc = np.zeros((10, 10, 5))
precision = np.zeros((10, 10, 5))
recall = np.zeros((10, 10, 5))
f1 = np.zeros((10, 10, 5))
kappa = np.zeros((10, 10, 5))
for i in range(10):
    for j in range(i+1, 10):
        data = np.compress(np.any([labels==i, labels==j], axis=0), fft_signals, axis=0)
        target = np.compress(np.any([labels==i, labels==j], axis=0), labels, axis=0)
        num = data.shape[0]
        data = data.reshape(num, -1)
        
        cv = ShuffleSplit(n_splits=5, test_size=.3, random_state=0)
        for k, (train_index, test_index) in enumerate(cv.split(data)):
            X_train = data[train_index]
            y_train = target[train_index]
            X_test = data[test_index]
            y_test = target[test_index]
            
            # PCA
            pca = PCA(n_components=20)
            pca.fit(X_train)
            X_train = pca.transform(X_train)
            X_test = pca.transform(X_test)
            # kNN
            clf = KNeighborsClassifier(n_neighbors=2, weights=gaussian)
            clf.fit(X_train, y_train)

            y_pred = clf.predict(X_test)
            acc[i, j, k] = ("%.3f" % accuracy_score(y_test, y_pred))
            acc[j, i, k] = acc[i, j, k]
            precision[i, j, k] = ("%.3f" % precision_score(y_test, y_pred, average="macro"))
            precision[j, i, k] = precision[i, j, k]
            recall[i, j, k] = ("%.3f" % recall_score(y_test, y_pred, average="macro"))
            recall[j, i, k] = recall[i, j, k]
            f1[i, j, k] = ("%.3f" % f1_score(y_test, y_pred, average="macro"))
            f1[j, i, k] = f1[i, j, k]
            kappa[i, j, k] = ("%.3f" % cohen_kappa_score(y_test, y_pred))
            kappa[j, i, k] = kappa[i, j, k]

In [7]:
def calc(pref_mat, name):
    print("========", name, "========")
    for i in range(5):
        print(">>> Cross Validation", i+1)
        #print("Number 0~10 max %s:" % name)
        #print(" ", np.max(pref_mat, axis=-1))
        print("0-10 %s:" % name, end=" ")
        mean = [float("%.2f" % (np.sum(li)/9)) for li in pref_mat[:, :, i]]
        print(mean)
        print("Average %s: %.2f" % (name, np.mean(mean)))
    print()

In [8]:
calc(acc, name="Accuracy")
calc(precision, name="Precision")
calc(recall, name="Recall")
calc(f1, name="F1 score")
calc(kappa, name="Kappa coef")

======== Accuracy ========
>>> Cross Validation 1
0-10 Accuracy: [0.86, 0.84, 0.85, 0.85, 0.86, 0.85, 0.85, 0.85, 0.85, 0.86]
Average Accuracy: 0.85
>>> Cross Validation 2
0-10 Accuracy: [0.85, 0.86, 0.86, 0.85, 0.85, 0.86, 0.85, 0.86, 0.84, 0.85]
Average Accuracy: 0.85
>>> Cross Validation 3
0-10 Accuracy: [0.85, 0.87, 0.85, 0.85, 0.84, 0.86, 0.85, 0.85, 0.85, 0.87]
Average Accuracy: 0.85
>>> Cross Validation 4
0-10 Accuracy: [0.85, 0.85, 0.86, 0.85, 0.85, 0.85, 0.85, 0.85, 0.86, 0.86]
Average Accuracy: 0.85
>>> Cross Validation 5
0-10 Accuracy: [0.85, 0.86, 0.85, 0.85, 0.86, 0.85, 0.85, 0.86, 0.86, 0.86]
Average Accuracy: 0.86

======== Precision ========
>>> Cross Validation 1
0-10 Precision: [0.89, 0.88, 0.89, 0.89, 0.89, 0.89, 0.89, 0.89, 0.89, 0.89]
Average Precision: 0.89
>>> Cross Validation 2
0-10 Precision: [0.88, 0.89, 0.89, 0.89, 0.89, 0.89, 0.89, 0.89, 0.88, 0.88]
Average Precision: 0.89
>>> Cross Validation 3
0-10 Precision: [0.88, 0.9, 0.89, 0.89, 0.88, 0.89, 0.89, 0.89,